In [ ]:
import sys
import os

sys.path.append('../')
from dataloader.MiddleburyLoader import myImageFloder
from notebooks.notebook_utils import visualize_sample, download_file_s3, list_s3

In [ ]:
def get_all_keys(disp_folder, img_folder, base_name):
    li = img_folder + '/left/{}.png'.format(base_name)
    ri = img_folder + '/right/{}.png'.format(base_name)
    ld = disp_folder + '/left/{}.pfm'.format(base_name)
    rd = disp_folder + '/right/{}.pfm'.format(base_name)
    return li, ri, ld, rd

def get_out_file(s3_key):
    s3_key_split = s3_key.split('/')
    out_folder = '../data/{}'.format('/'.join(s3_key_split[1:-1]))
    os.makedirs(out_folder, exist_ok=True)
    out_file = os.path.join(out_folder, '{}'.format(s3_key_split[-1]))
    return out_file
    
def walk_through_dataset(dataset_list):
    for example in dataset_list:
        left_key = example['Key']
        if 'left' not in left_key:
            continue
        base_name = os.path.basename(left_key)[:-4]
        disp_folder = '/'.join(left_key.split('/')[0:-2])
        img_folder = disp_folder.replace('disparity', 'frames_cleanpass')
        li, ri, ld, rd = get_all_keys(disp_folder, img_folder, base_name)
        print(li)
        print(ri)
        print(ld)
        print(rd)
        
        all_left = [get_out_file(li)]
        all_right = [get_out_file(ri)]
        all_left_disp = [get_out_file(ld)]
        all_right_disp = [get_out_file(rd)]
        
        download_file_s3(li, all_left[0])
        download_file_s3(ri, all_right[0])
        download_file_s3(ld, all_left_disp[0])
        download_file_s3(rd, all_right_disp[0])
        
        rand_scale = [0.9, 2.4]
        loader_scene = myImageFloder(all_left, all_right, all_left_disp,
                                     right_disparity=all_right_disp, 
                                     rand_scale=rand_scale, order=2)
        
        for (img_l, img_r, disp_l) in loader_scene:
            visualize_sample(img_l, img_r, disp_l)
            yield

# Monkaa

In [ ]:
train_list = list_s3('hdsm/sceneflow/monkaa/disparity/', delim='')['Contents']

In [ ]:
train_iter = walk_through_dataset(train_list)

In [ ]:
next(train_iter)

# Driving

In [ ]:
train_list = list_s3('hdsm/sceneflow/driving/disparity/', delim='')['Contents']

In [ ]:
train_iter = walk_through_dataset(train_list)

In [ ]:
next(train_iter)